## Access images

In [1]:
import astroquery
print(astroquery.__version__)

0.4.11.dev10140


https://stackoverflow.com/questions/59699193/how-to-obtain-2d-cutout-of-an-image-from-a-skycoord-position

In [23]:
# For Angle conversion
from astropy.coordinates import Angle
import astropy.units as u
from astropy.coordinates import SkyCoord
import pandas as pd
import numpy as np
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.nddata import Cutout2D

In [24]:
from IPython.display import Image, display

## Configuration

### Target in ECDFS

https://arxiv.org/pdf/1104.0931

In [25]:
fig_table1 = "input_figs/table1_gemcandidates.png"
fig_table2 = "input_figs/table2_gemcandidates.png"

In [33]:
Image(url= fig_table1,width=800)

In [32]:
Image(url= fig_table2,width=500)

In [36]:
#15422 44 03:32:38.21 –27:56:53.2 
ra1 = "03:32:38.21 hours"
dec1 = "-27:56:53.2 degrees"
tile1 = 44

#34244 94 03:32:06.45 –27:47:28.6 
ra2 = "03:32:06.45 hours"
dec2 = "-27:47:28.6 degrees"
tile2 = 94

#40173 35 03:33:19.45 –27:44:50.0 
ra3 = "03:33:19.45 hours"
dec3 = "-27:44:50.0 degrees"
tile3 = 35

#43242 45 03:31:55.35 –27:43:23.5 
ra4 = "03:31:55.35 hours"
dec4 = "-27:43:23.5 degrees"
tile4 = 45

#46446 47 03:31:35.94 –27:41:48.2 
ra5 = "03:31:35.94 hours"
dec5 = "-27:41:48.2 degrees"
tile5 = 47

#12589 03:31:24.89 −27:58:07.0
ra6 = "03:31:24.89 hours"
dec6 = "-27:58:07.0 degrees"
tile6 = 17

#43797 03:31:31.74 −27:43:00.8 
ra7 = "03:31:31.74 hours"
dec7 = "-27:43:00.8 degrees"
tile7 = 47

#28294 03:31:50.54 −27:50:28.4 
ra8 = "03:31:50.54 hours"
dec8 = "-27:50:28.4 degrees"
tile8 = 33

#36857 03:31:53.24 −27:46:18.9
ra9 = "03:31:53.24 hours"
dec9 = "-27:46:18.9 degrees"
tile9 = 38

#36714 03:32:59.78 −27:46:26.4 
ra10 = "03:32:59.78 hours"
dec10 = "-27:46:26.4 degrees"
tile10 = 37


In [37]:
lsstcomcam_targets = {}
# high rank
lsstcomcam_targets["ECDFS_G15422"] = {"field_name": "GEMS-15422", "ra": 53.159208333333325, "dec": -27.94811111111111,"tile":tile1}
lsstcomcam_targets["ECDFS_G34244"] = {"field_name": "GEMS-34244", "ra": 53.02687499999999 , "dec": -27.79127777777778,"tile":tile2}
lsstcomcam_targets["ECDFS_G40173"] = {"field_name": "GEMS-40173", "ra": 53.33104166666666 , "dec": -27.747222222222224,"tile":tile3}
lsstcomcam_targets["ECDFS_G43242"] = {"field_name": "GEMS-43242", "ra": 52.980624999999996 , "dec": -27.72319444444444,"tile":tile4}
lsstcomcam_targets["ECDFS_G46446"] = {"field_name": "GEMS-46446", "ra": 52.89975 , "dec": -27.696722222222224,"tile":tile5}

# low rank
lsstcomcam_targets["ECDFS_G12589"] = {"field_name": "GEMS-12589", "ra": 52.85370833333333, "dec": -27.96861111111111,"tile":tile6}
lsstcomcam_targets["ECDFS_G43797"] = {"field_name": "GEMS-43797", "ra": 52.88224999999999, "dec": -27.71688888888889,"tile":tile7}

lsstcomcam_targets["ECDFS_G28294"] = {"field_name": "GEMS-28294", "ra": 52.960583333333325 , "dec": -27.84122222222222,"tile":tile8}
lsstcomcam_targets["ECDFS_G6857"] = {"field_name": "GEMS-6857", "ra": 52.97183333333333 , "dec": -27.771916666666666,"tile":tile9}
lsstcomcam_targets["ECDFS_G36714"] = {"field_name": "GEMS-36714", "ra": 53.249083333333324, "dec": -27.773999999999997,"tile":tile10}


In [38]:
df = pd.DataFrame(lsstcomcam_targets).T

In [39]:
# candidates
#key = "ECDFS_G15422"
key = "ECDFS_G34244"
#key = "ECDFS_G40173"
#key= "ECDFS_G43242"
#key= "ECDFS_G46446"

# unknown
#key = "ECDFS_G12589"
#key = "ECDFS_G43797"
#key = "ECDFS_G28294"
#key = "ECDFS_G6857"
#key = "ECDFS_G36714"

the_target = lsstcomcam_targets[key]
target_ra = the_target["ra"]
target_dec = the_target["dec"]
target_name = the_target["field_name"]
tile_num = the_target["tile"]

target_title = (
    the_target["field_name"] + f" (ra,dec) = ({target_ra:.2f},{target_dec:.2f}) "
)

In [40]:
tile_num

94

### Access to remote files

In [7]:
# Charger l'image

# Load the FITS file and create a SkyCoord object for 
# the cutout center

url = f"http://archive.stsci.edu/pub/hlsp/gems/z_mk1/h_gems_z{tile_num}_mk1.fits"
hdul = fits.open(url)

In [8]:
print(hdul.info())

Filename: /Users/dagoret/.astropy/cache/download/url/3119fbe9c74e679967daa7b90812391d/contents
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     440   (7060, 7252)   float32   
None


In [9]:
hdu = hdul[0]
data = hdu.data
header = hdu.header
wcs = WCS(header)

### Check the pixel scale

In [10]:
pixel_scale_matrix = wcs.pixel_scale_matrix

# Norme des vecteurs de la matrice (en degrés/pixel)
scale_x = np.linalg.norm(pixel_scale_matrix[0])
scale_y = np.linalg.norm(pixel_scale_matrix[1])

# Moyenne des deux axes
scale_deg = (scale_x + scale_y) / 2

# Conversion en arcsec/pixel
scale_arcsec = scale_deg * 3600

print(f"Pixel scale ≈ {scale_arcsec:.3f} arcsec/pixel")

Pixel scale ≈ 0.030 arcsec/pixel


### Build the target coordinates

In [11]:
# Exemple : coordonnées au centre du champ GEMS
coord = SkyCoord(ra=target_ra, dec=target_dec, unit=(u.deg, u.deg))

## Make the cutout

In [12]:
cut_im_path = f"cutouthst_{key}.fits"

In [13]:
cutout_header = hdu.header.copy()

# Create a 2D WCS specifically for the spatial cutout
wcs_full = WCS(cutout_header)
wcs_2d = wcs_full.celestial

# Extract only the 2D spatial slice of the data
data_2d = data
size = (128, 128)

# Create the cutout with the 2D WCS
cutout = Cutout2D(data_2d, position=coord, size=size, wcs=wcs_2d)

cutout_wcs_to_header = cutout.wcs.to_header()
# Essential line to retain other axes (e.g frequency and Stokes).
cutout_wcs_to_header["WCSAXES"] = hdu.header['NAXIS']

cutout_header.update(cutout_wcs_to_header)

# Save the cutout data with the updated header
hdu_cutout = fits.PrimaryHDU(data=cutout.data, header=cutout_header)
try:
    hdu_cutout.writeto(cut_im_path, overwrite=False)
except OSError:
    print("Output file already exists.")


NoOverlapError: Arrays do not overlap.